# A basic training loop

This notebook builds upon the work of the [previous notebook](001a_nn_basics.ipynb) in which we created a simlple training loop (including calculating the loss on a validation set) and then a 3-layer CNN using PyTorch's Sequential class.

Here, we will

## From the last notebook...

In [ ]:
#export
import pickle, gzip, torch, math, numpy as np, torch.nn.functional as F
from pathlib import Path
from IPython.core.debugger import set_trace
from torch import nn, optim, tensor

from torch.utils.data import TensorDataset, Dataset, DataLoader
from dataclasses import dataclass
from typing import Any, Collection, Callable
from functools import partial, reduce
from numbers import Number

The data was downloaded in section 1.1 of the [previous notebook](001a_nn_basics.ipynb) so make sure you have run that code before you continue.

In [ ]:
DATA_PATH = Path('data')
PATH = DATA_PATH/'mnist'

with gzip.open(PATH/'mnist.pkl.gz', 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

x_train,y_train,x_valid,y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))

After creating our training and validation sets, we print out the min and max to get a sense for the range of feature values. It is always a good idea to inspect your data. In the case of the MNIST dataset, the x-values for each training example correspond to pixel values that, range from 0 to ~1.

In [ ]:
x_train.min(),x_train.max()

In [ ]:
tensor(x_train)

`bs` stands for batch size and `lr` stands for learning rate. [Here is a reference](https://github.com/fastai/fastai_v1/blob/master/docs/abbr.md) for these and other abbreviations used as variable names. The fast.ai library differs from PEP 8 and instead follows conventions developed around the [APL](https://en.wikipedia.org/wiki/APL_\(programming_language\)) / [J](https://en.wikipedia.org/wiki/J_\(programming_language\)) / [K](https://en.wikipedia.org/wiki/K_\(programming_language\)) programming languages (all of which are centered around multi-dimensional arrays), which are more concise and closer to math notation. [Here is a more detailed explanation](https://github.com/fastai/fastai/blob/master/docs/style.md) of the fast.ai style guide.

In [ ]:
bs=64
epochs = 2
lr=0.2

PyTorch's [TensorDataset](https://pytorch.org/docs/stable/_modules/torch/utils/data/dataset.html#TensorDataset) is a Dataset wrapping tensors. It gives us a way to iterate, index, and slice along the first dimension of a tensor. This will make it easier to access both the independent and dependent variables in the same line as we train.

In [ ]:
train_ds = TensorDataset(x_train, y_train)
valid_ds = TensorDataset(x_valid, y_valid)

We are using the same `loss_batch`, `fit`, and `Lambda` as were defined in the previous notebook:

In [ ]:
#export
def is_listy(x)->bool: return isinstance(x, (tuple,list))

def loss_batch(model, xb, yb, loss_fn, opt=None):
    if not is_listy(xb): xb = [xb]
    if not is_listy(yb): yb = [yb]
    loss = loss_fn(model(*xb), *yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    return loss.item(), len(xb)

In [ ]:
#export
def fit(epochs, model, loss_fn, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb,yb in train_dl: loss,_ = loss_batch(model, xb, yb, loss_fn, opt)

        model.eval()
        with torch.no_grad():
            losses,nums = zip(*[loss_batch(model, xb, yb, loss_fn)
                                for xb,yb in valid_dl])
        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)

        print(epoch, val_loss)

In [ ]:
#export
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func=func
        
    def forward(self, x): return self.func(x)

## Simplify nn.Sequential layers

As a reminder, our 3-layer CNN from the previous notebook was defined:

```
model = nn.Sequential(
    Lambda(lambda x: x.view(-1,1,28,28)),
    nn.Conv2d(1,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AvgPool2d(4),
    Lambda(lambda x: x.view(x.size(0),-1))
)
```

Let's refactor this a bit to make it more readable, and to make the components more reusable:

In [ ]:
#export
def ResizeBatch(*size): return Lambda(lambda x: x.view((-1,)+size))
def Flatten(): return Lambda(lambda x: x.view((x.size(0), -1)))
def PoolFlatten(): return nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten())

Using our newly defined `ResizeBatch`, `Flatten`, and `PoolFlatten`, we can instead now define the same networks as:

In [ ]:
model = nn.Sequential(
    ResizeBatch(1,28,28),
    nn.Conv2d(1,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    PoolFlatten()
)

Note that we will nearly always use small kernels of size 3 due to the reasons presented in section 2.3 in [this paper](https://arxiv.org/pdf/1409.1556.pdf) (a few small kernels achieve a receptive field of the same dimension as one bigger kernel while at the same time achieving increased discriminative power and using fewer parameters). 

We will use the same `get_data method` as defined in the previous notebook:

In [ ]:
def get_data(train_ds, valid_ds, bs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True),
            DataLoader(valid_ds, batch_size=bs*2))

train_dl,valid_dl = get_data(train_ds, valid_ds, bs)

**Set loss function**

[Here](https://rdipietro.github.io/friendly-intro-to-cross-entropy-loss/) is tutorial explaining why cross entropy is a resonable loss function for classifciation tasks.

In [ ]:
loss_fn = F.cross_entropy

**Set optimizer**

We stick with stochastic gradient descent without momentum as our optimizer. This is a basic optimizer and it is [easy to understand](http://ruder.io/optimizing-gradient-descent/index.html#stochasticgradientdescent). We will move into better optimizers as we go forward.

In [ ]:
opt = optim.SGD(model.parameters(), lr=lr)

**Test our loss function**

We try out our loss function on one batch of X features and y targets to make sure it's working correctly.

In [ ]:
loss_fn(model(x_valid[0:bs]), y_valid[0:bs])

**Fit**

Everything looks ready, we call the fit function we developed earlier for two epochs to confirm that the model learns.

In [ ]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

## Transformations

We are going to refactor some of the data transformations out of the network and into a pipeline that is applied to the data being fed into the Dataloders.

This is more flexible, simplifies the model, and will be useful later when we want to apply additional transformations for things like data augmentation.

**Define transformations**

In this example our only transformation will be *mnist2image*. This is a utility function to reshape our features into 28x28 arrays.

X is a batch of features where the first dimension is the number of samples in the batch and the remaining dimensions define the shape of the training example. y is the target variable to be learned, in this case it an integer representing one of 10 image classes.

With MNIST data, the X features start out as a 1x784 vector and we want to convert the features to 1x28x28 images (see line 62). This helper function does that for an entire batch of features.

In [ ]:
def mnist2image(b): return b.view(1,28,28)

In [ ]:
#export
@dataclass
class DatasetTfm(Dataset):
    ds: Dataset
    tfm: Callable = None
        
    def __len__(self): return len(self.ds)
    
    def __getitem__(self,idx):
        x,y = self.ds[idx]
        if self.tfm is not None: x = self.tfm(x)
        return x,y

In [ ]:
train_tds = DatasetTfm(train_ds, mnist2image)
valid_tds = DatasetTfm(valid_ds, mnist2image)

In [ ]:
def get_data(train_ds, valid_ds, bs):
    return (DataLoader(train_ds, bs,   shuffle=True),
            DataLoader(valid_ds, bs*2, shuffle=False))

In [ ]:
train_dl,valid_dl = get_data(train_tds, valid_tds, bs)

We make some checks to make sure that *mnist2image* is working correctly:
1. The input and output shapes are as expected
2. The input and output data (features) are the same

In [ ]:
x,y = next(iter(valid_dl))

In [ ]:
valid_ds[0][0].shape, x[0].shape

In [ ]:
torch.allclose(valid_ds[0][0], x[0].view(-1))

## Refactor network

**Define layer types and loop over them**

When use a layer type more than once in a contiguous fashion (one after the other), it makes sense to define a function for that layer type and then use that function to build our model function. 

That is what we do here with *conv2_relu* with which we avoid the three subsequent lines of code in line 12 (this saving becomes more significant in deeper networks).

In [ ]:
#export
def conv2_relu(nif, nof, ks, stride):
    return nn.Sequential(nn.Conv2d(nif, nof, ks, stride, padding=ks//2), nn.ReLU())

def simple_cnn(actns, kernel_szs, strides):
    layers = [conv2_relu(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(PoolFlatten())
    return nn.Sequential(*layers)

In [ ]:
def get_model():
    model = simple_cnn([1,16,16,10], [3,3,3], [2,2,2])
    return model, optim.SGD(model.parameters(), lr=lr)

In [ ]:
model,opt = get_model()

In [ ]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

## CUDA

**Run in GPU and add progress bar**

To run our Pytorch networks in the GPU we have to specify it in the code. This is done by setting *torch.device('cuda')*. We will also add a progress bar to keep track of the progress during training. This we accomplish with [fast_progress](https://github.com/fastai/fast_progress) package.

We integrate both these features into a custom Dataloader which we build on top of the Pytorch Dataloader.

In [ ]:
#export
default_device = torch.device('cuda')

def to_device(b, device):
    if device is None: device=default_device
    if is_listy(b): return [to_device(o, device) for o in b]
    return b.to(device)

@dataclass
class DeviceDataLoader():
    dl: DataLoader
    device: torch.device

    def __len__(self): return len(self.dl)
    def proc_batch(self,b): return to_device(b, self.device)

    def __iter__(self):
        self.gen = map(self.proc_batch, self.dl)
        return iter(self.gen)

    @classmethod
    def create(cls, *args, device=default_device, **kwargs): return cls(DataLoader(*args, **kwargs), device=device)

In [ ]:
def get_data(train_ds, valid_ds, bs):
    return (DeviceDataLoader.create(train_ds, bs,   shuffle=True, num_workers=2),
            DeviceDataLoader.create(valid_ds, bs*2, shuffle=False, num_workers=2))

In [ ]:
train_dl,valid_dl = get_data(train_tds, valid_tds, bs)

In [ ]:
def get_model():
    model = simple_cnn([1,16,16,10], [3,3,3], [2,2,2]).to(default_device)
    return model, optim.SGD(model.parameters(), lr=lr)

In [ ]:
model,opt = get_model()

In [ ]:
x,y = next(iter(valid_dl))

In [ ]:
x.type(),y.type()

In [ ]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

## Learner

**Define learner**

Finally, we are missing a learner class to close the gap between our loaded data and our model. The learner class will recieve our loaded data (after transformations) and the model and we will be able to call fit on it to start the training phase.

In [ ]:
#export
@dataclass
class DataBunch():
    train_dl:DataLoader
    valid_dl:DataLoader
    device:torch.device=None

    @classmethod
    def create(cls, train_ds, valid_ds, bs=64, train_tfm=None, valid_tfm=None, device=None, **kwargs):
        return cls(DeviceDataLoader.create(DatasetTfm(train_ds,train_tfm), bs, shuffle=True, device=device,**kwargs),
                   DeviceDataLoader.create(DatasetTfm(valid_ds, valid_tfm), bs*2, shuffle=False, device=device, **kwargs),
                   device=device)

class Learner():
    def __init__(self, data, model):
        self.data,self.model = data,to_device(model, data.device)

    def fit(self, epochs, lr, opt_fn=optim.SGD):
        opt = opt_fn(self.model.parameters(), lr=lr)
        loss_fn = F.cross_entropy
        fit(epochs, self.model, loss_fn, opt, self.data.train_dl, self.data.valid_dl)

In [ ]:
data = DataBunch.create(train_ds, valid_ds, bs=bs, train_tfm=mnist2image, valid_tfm=mnist2image)
model = simple_cnn([1,16,16,10], [3,3,3], [2,2,2])
learner = Learner(data, model)
opt_fn = partial(optim.SGD, momentum=0.9)

In [ ]:
learner.fit(1, lr/5, opt_fn=opt_fn)

In [ ]:
learner.fit(2, lr, opt_fn=opt_fn)
learner.fit(1, lr/5, opt_fn=opt_fn)